# Network building

Based on artists sharing a common announcement/exhibition: using *announcements.json* / *announcements_contemporary.json* 

## Initial steps

In [1]:
import scraping_functions

In [2]:
with open('data/artists_contemporary_cleaned_v1.txt', 'r', encoding='utf-8') as f:
    artists_contemporary = f.read().splitlines()

with open('data/artists_cleaned_v1.txt', 'r', encoding='utf-8') as f:
    artists = f.read().splitlines()

## 1) Contemporary announcements only:

**TODO**:

In [ ]:
import graph-tool.all as gt